## 1. 예술 경영 빅데이터 공모전 데이터 수집 (크롤링)  

출처 : Sometrend (소셜 미디어 분석)

In [2]:
# 필요한 라이브러리 불러오기
import pandas as pd  

pd.set_option('display.max_columns', None)

from selenium import webdriver  
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC  
from bs4 import BeautifulSoup  
from pandas import DataFrame  
import time  
import requests 
import re  

import urllib.request    

In [4]:
# 데이터 읽어오기 
data = pd.read_csv('G:\내 드라이브\Ajou univ 3-2\예술경영 빅데이터 공모전\이용자_관점_크롤링_전처리_data (version 1).csv', index_col = 0)

In [5]:
data.head(3)

,전송사업자코드,전송사업자명,공연시설코드,시설특성,개관연도,주소,편의시설_레스토랑 여부,편의시설_카페 여부,편의시설_편의점 여부,편의시설_놀이방 여부,편의시설_수유실 여부,장애인시설_주차장 여부,장애인시설_화장실 여부,장애인시설_경사로 여부,장애인시설_전용엘리베이터 여부,주차시설_자체 여부,주차시설_공영 여부,공연장코드,좌석수,장애인석,무대시설_오케스트라피트 여부,무대시설_연습실 여부,무대시설_분장실 여부,무대시설_무대넓이,공연코드,공연회차,공연일시,입장권고유번호,예매/취소일시,예매/취소구분,예매/취소매수,예매/취소금액,예매/취소방식코드,예매/취소방식명(관리시스템),예매/취소방식명(전송처),결제수단코드,결제수단명(관리시스템),결제수단명(전송처),할인금액,할인종류코드,할인종류명(관리시스템),할인종류명(전송처),장당금액,성별,연령,공연시작일자,공연종료일자,소요시간,장르명,세부장르명,공연지역명,출연진내용,제작진내용,기획제작사명,원작자명,극작가명,수상실적,관람연령,아동공연 여부,축제 여부,내한공연 여부,오픈런 여부,판매시작일시,판매종료일시,좌석등급,단독판매여부,판매좌석수,판매페이지URL,기획_제작사명
0,SE000102,티켓링크,FC223315,공공(문예회관),1975.0,대구광역시 중구 태평로 141 (태평로2가),N,Y,N,N,Y,Y,Y,Y,Y,Y,N,FC223315-01,1284,14,Y,Y,Y,344,PF314870,1,2022-06-21 19:30,1431527281_1,2022-06-10 13:40,1,1,0,99,기타,기획사/구단,1,현금,현금,0,99,기타,NaN,0,0,0,2022-06-21,2022-06-21,1시간 30분,클래식,기악,경상도,"이광호, 이정아, 백민아, 오수진 등",NaN,"한국기독교총연합회(주최), CTS기독교TV(주관)",NaN,NaN,NaN,만 7세 이상,N,N,N,N,2022-06-09 0:00,2022-06-20 17:00,전석무료(0),N,1석,http://www.ticketlink.co.kr/product/38972,한국기독교총연합회
1,SE000102,티켓링크,FC223315,공공(문예회관),1975.0,대구광역시 중구 태평로 141 (태평로2가),N,Y,N,N,Y,Y,Y,Y,Y,Y,N,FC223315-01,1284,14,Y,Y,Y,344,PF313959,1,2022-06-22 19:30,1430846151_1,2022-05-25 15:40,1,1,0,99,기타,기획사/구단,1,현금,현금,0,99,기타,NaN,0,0,0,2022-06-22,2022-06-22,NaN,클래식,성악,경상도,"이선경, 김소정, 한혜민, 박소연, 박인아, 김서준, 김남수 등",이선경 등,"명음클래식(주최), 명음클래식(주관)",NaN,NaN,NaN,만 7세 이상,N,N,N,N,2022-05-18 0:00,2022-06-21 17:00,전석(20000),N,1석,http://www.ticketlink.co.kr/product/38712,명음클래식
2,SE000201,인터파크,FC223315,공공(문예회관),1975.0,대구광역시 중구 태평로 141 (태평로2가),N,Y,N,N,Y,Y,Y,Y,Y,Y,N,FC223315-01,1284,14,Y,Y,Y,344,PF314585,1,2022-06-24 19:30,T0896496779,2022-06-07 11:15,1,1,0,3,현장,대구콘서트하우스,1,현금,현금,0,99,기타,NaN,0,0,0,2022-06-24,2022-06-24,1시간,클래식,기악,경상도,"류명우, 소정환, 송경민, 유하준, 최미지, 박시인 등",NaN,"대구시립예술단(주최), 대구시립교향악단(주관)",NaN,NaN,NaN,만 7세 이상,N,N,N,N,2022-05-31 0:00,2022-06-24 16:30,전석무료(0),Y,1254석,http://ticket.interpark.com/Ticket/Goods/Goods...,대구시립예술단


In [8]:
# 데이터 전처리 및 통계확인 

data['기획_제작사명'] = data['기획_제작사명'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data['기획_제작사명'] = data['기획_제작사명'].str.replace("&", "")  
data['기획_제작사명'] = data['기획_제작사명'].str.replace(" ", "")  
data['기획_제작사명'] = data['기획_제작사명'].str.replace(",", "")    


data['기획_제작사명'].nunique()
#410개 정도 unique한 열있음 
data['기획_제작사명'].value_counts()

In [12]:
# 기획_제작사명별로 중복 행 제거 
# ignore_index = True 면 인덱스 다시 정렬 
data2 = data.drop_duplicates(['기획_제작사명'], keep= 'first')  
data2['기획_제작사명'].nunique()

In [33]:
## 기획_제작사별 긍부정 크롤링  

driver = webdriver.Chrome('C:/Users/wlsdk/chromedriver.exe')  


driver.get('https://some.co.kr/')  


driver.implicitly_wait(3)   



driver.find_element_by_class_name('btn-login').click()
driver.find_element_by_name('username').send_keys('아이디')  
driver.find_element_by_name('password').send_keys( '비밀번호' )  
driver.find_element_by_id('kc-login').click()  

time.sleep(3)

# 1. 소셜분석  누르기

driver.find_element_by_css_selector('#naviTitle > li:nth-child(1) > a').click()

# 2. 연관 분석 누르기

driver.find_element_by_xpath('/html/body/div[4]/div/header/div/nav/ul/li[1]/ul/li[3]/a').click()

time.sleep(3)

sentiment = []


# 검색키워드에 기획_제작사명별로 입력하여 긍부정 정도 text 추출

for keyword in data2['기획_제작사명']:

      
      driver.find_element_by_class_name('input-keyword-text').click()

      time.sleep(3)

      search2 = driver.find_element_by_css_selector('#analysisOptionWindowArea > form > fieldset > div > div.modal-search-header > div > input')

      search2.clear()
      search2.send_keys(keyword)
      search2.send_keys(Keys.ENTER)  
           
      driver.find_element_by_xpath('//*[@id="6Mlabel"]').click()

      time.sleep(3)

      try:
            driver.find_element_by_xpath('//*[@id="6Mlabel"]').click()
            sen = driver.find_element_by_css_selector('#rateOfSentiment') 
            sentiment.append(sen.text) 

      except:
            sentiment.append('NA')
            print('error')

In [34]:
# 최종 추출한 감정
sentiment

['부정 56%',
 '중립',
 '긍정 84%',
 '중립',
 '긍정 89%',
 '중립',
 '중립',
 '긍정 76%',
 '긍정 70%',
 '중립',
 '긍정 87%',
 '긍정 93%',
 '중립',
 '중립',
 '긍정 88%',
 '긍정 89%',
 '긍정 100%',
 '긍정 88%',
 '중립',
 '긍정 63%',
 '부정 100%',
 '긍정 82%',
 '중립',
 '중립',
 '긍정 80%',
 '중립',
 '긍정 77%',
 '긍정 54%',
 '중립',
 '긍정 67%',
 '긍정 83%',
 '중립',
 '긍정 83%',
 '중립',
 '중립',
 '긍정 85%',
 '긍정 100%',
 '긍정 96%',
 '긍정 76%',
 '긍정 100%',
 '긍정 91%',
 '중립',
 '긍정 72%',
 '부정 58%',
 '긍정 100%',
 '중립',
 '중립',
 '긍정 76%',
 '중립',
 '중립',
 '긍정 77%',
 '부정 61%',
 '긍정 100%',
 '중립',
 '긍정 91%',
 '긍정 91%',
 '긍정 88%',
 '긍정',
 '중립',
 '',
 '중립',
 '긍정 75%',
 '긍정 100%',
 '부정 73%',
 '긍정 79%',
 '중립',
 '중립',
 '긍정 89%',
 '긍정 79%',
 '중립',
 '긍정 88%',
 '중립',
 '긍정 100%',
 '부정 75%',
 '긍정 83%',
 '긍정 80%',
 '중립',
 '긍정 92%',
 '중립',
 '긍정 88%',
 '중립',
 '긍정 67%',
 '긍정 88%',
 '긍정 90%',
 '긍정 100%',
 '중립',
 '긍정 94%',
 '긍정 91%',
 '중립',
 '긍정 83%',
 '긍정 83%',
 '긍정 82%',
 '긍정 84%',
 '긍정 92%',
 '긍정 81%',
 '긍정 100%',
 '긍정 93%',
 '중립',
 '긍정 100%',
 '긍정 80%',
 '중립',
 '긍정 88%',
 '중립',
 '긍정 7

In [35]:
data2['기획_제작사명_긍부정'] = sentiment

C:\Users\wlsdk\AppData\Local\Temp/ipykernel_2836/3550428256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['기획_제작사명_긍부정'] = sentiment


In [37]:
# 긍부정 열을 포함한 데이터 csv파일 
data2.to_csv('예술경영_빅데이터공모전.ver진영(긍부정).csv', encoding = 'utf-8-sig')

## 2. 데이터 수집 - ESG 경영 키워드를 통한 이슈화 크롤링 

In [ ]:
data3 = pd.read_csv('G:\내 드라이브\Ajou univ 3-2\예술경영 빅데이터 공모전\예술경영_빅데이터공모전.ver진영(긍부정).csv', index_col = 0)

In [ ]:
## ESG키워드 크롤링  

driver = webdriver.Chrome('C:/Users/wlsdk/chromedriver.exe')  


driver.get('https://some.co.kr/')  


driver.implicitly_wait(3)   



driver.find_element_by_class_name('btn-login').click()
driver.find_element_by_name('username').send_keys('wlsdk2450@ajou.ac.kr')  
driver.find_element_by_name('password').send_keys( 'gkdlqkr02!' )  
driver.find_element_by_id('kc-login').click()  

time.sleep(3)

# 1. 소셜분석  누른다 
driver.find_element_by_css_selector('#naviTitle > li:nth-child(1) > a').click()

# 2. 연관어분석 누른다

driver.find_element_by_xpath('/html/body/div[4]/div/header/div/nav/ul/li[1]/ul/li[2]/a').click()

time.sleep(3)

association_list = []
keywords = data3['기획_제작사명']

# 검색어의 기획_제작사명별 ESG에 해당하는 키워드 크롤링 
for keyword in keywords:

      i =  data3[data3['기획_제작사명'] == keyword].index[0]

      driver.find_element_by_class_name('input-keyword-text').click()

      time.sleep(3)  

      search2 = driver.find_element_by_css_selector('#analysisOptionWindowArea > form > fieldset > div > div.modal-search-header > div > input')

      search2.clear()
      search2.send_keys(keyword)
      search2.send_keys(Keys.ENTER)  
            # 5. 카테고리에서 전체를 지우고, class = sider
      driver.find_element_by_xpath('//*[@id="6Mlabel"]').click()

      time.sleep(3)

      try:
            driver.find_element_by_css_selector('#categoryCheckItems > div.category-select-header > div > label > span.slider').click()
            driver.find_element_by_css_selector('#categoryCheckItems > div.category-select-header > div > label > span.slider').click()
      

            #  정부조직, 경제사회, 자연환경 필터링만 


            time.sleep(3)

            driver.find_element_by_css_selector('#categoryCheckItems > div.category-select-body > ul > li:nth-child(2) > div.one-depth-caterory > div > label > span.slider').click()
            driver.find_element_by_css_selector('#categoryCheckItems > div.category-select-body > ul > li:nth-child(6) > div.one-depth-caterory > div > label > span.slider').click()
            driver.find_element_by_css_selector('#categoryCheckItems > div.category-select-body > ul > li:nth-child(8) > div.one-depth-caterory > div > label > span.slider').click()

      



            driver.find_element_by_css_selector('#categorySkeletonTarget > div.comp-category-select-box > button').click()
       
      
            association_word = []
      

            time.sleep(3)


            elems = driver.find_elements_by_css_selector('#associationWordCloud > svg > g.canvas > g.association_node') 

            for elem in elems:
                association_word.append(elem.text)

            association_word = list(association_word)
            association_list.append(association_word)


      except:
            association_list.append('NA')
            print('error')

In [ ]:
data3['기획_제작사_ESG키워드'] = association_list

In [ ]:
data3.head()

In [ ]:
data3.to_csv('예술경영_빅데이터공모전.ver진영(ESG키워드포함).csv', encoding ='utf-8-sig' )

In [ ]:
data4 = pd.read_csv('G:\내 드라이브\Ajou univ 3-2\예술경영 빅데이터 공모전\예술경영_빅데이터공모전.ver진영(ESG키워드포함).csv')
data4.head()

In [ ]:
# 기획제작사별 ESG키워드가 존재하면 0, 존재하지 않으면 1로 새로운 값 부여 
data4['ESG키워드_유무'] = np.where(data4['기획_제작사_ESG키워드'].isnull(), 0, 1)  

data4['ESG키워드_유무'].value_counts()  


In [ ]:
# 최종 데이터 수집 파일 저장 
data4.to_csv('예술경영_빅데이터공모전.ver크롤링최종(박진영).csv', encoding= 'utf-8-sig' )